<a href="https://colab.research.google.com/github/dlagur/ProjectHanium/blob/main/KoBERT_%EC%9E%90%EA%B8%B0%EC%86%8C%EA%B0%9C%EC%84%9C_%EC%A7%81%EB%AC%B4%EC%9A%94%EA%B5%AC%EC%97%AD%EB%9F%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 데이터 불러오기

In [2]:
import pandas as pd

data = pd.read_csv('/content/keyword_정제.csv')

In [3]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-t1dcgk8t
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-t1dcgk8t
  Using cached transformers-4.20.1-py3-none-any.whl (4.4 MB)
  Using cached tokenizers-0.12.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2


In [6]:
# KoBERT
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [7]:
# Transformer
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
# GPU 사용
# device = torch.device('cuda:0')

In [9]:
# BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [10]:
!wget -O .cache/ratings_train.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
!wget -O .cache/ratings_test.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt

--2022-07-18 09:25:36--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.58.110
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.58.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘.cache/ratings_train.txt’

.cache/ratings_trai 100%[===================>]  13.95M  16.2MB/s    in 0.9s    

2022-07-18 09:25:37 (16.2 MB/s) - ‘.cache/ratings_train.txt’ saved [14628807/14628807]

--2022-07-18 09:25:37--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.58.110
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.58.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (

## 전처리

In [11]:
dataset_train = nlp.data.TSVDataset(".cache/ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset(".cache/ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

In [12]:
data

,content,label
0,뉴욕 주민 뉴욕 월가 트레이더 유튜버 미국 펜실베이니아대와튼스쿨 졸업 전 맥킨지 전...,1
1,스타뉴스 문완식기자원 더걸스 솔로아티스 트레이블유빈은 뛰어난 전략가 후 엠아이아티스...,1
2,방송화면 캡쳐장 우영기자아티스트 겸유빈 이후 엠아이를 통해 숨어 있던 자신의 내면과...,1
3,안철수 후보 단일화 제안 철회 벼랑끝 협상 방법일 수도 안철수 현지지율로는 소멸의 ...,1
4,일 채널 후 엠아이 출연 채널 후 엠아이뉴스 서울 뉴스윤효정기자 아티스트 겸유빈이 ...,1
...,...,...
966,금융 핫 amp 뉴 경기 광주 지역농협 직원 억 횡령 적발 카카오페이 월 거래액 조...,5
967,대만 통일 방해 말라 평화 유지 중요 블링컨 러 우크라 침공 규탄 해야 언급 없어 ...,5
968,글로벌 CEO 도요다 아키오 도요타 CEO 모든 사고에 유감 눈물의 사과 서 차량 ...,5
969,이공계 인재 양성에도 강점 브랭섬홀 아시아 브랭섬홀 아시아 학생들이 디자인 테크놀로...,5


In [13]:

data_list = []
for q, label in zip(data['content'], data['label'])  :
    pro_data = []
    pro_data.append(str(q))
    pro_data.append(str(label))

    data_list.append(pro_data)

In [14]:
print(data_list[0])
print(data_list[-1])

['뉴욕 주민 뉴욕 월가 트레이더 유튜버 미국 펜실베이니아대와튼스쿨 졸업 전 맥킨지 전시티그룹 전모 건 근무 뉴욕 주민의 진짜 미국식 주식투자 저자 사진 고운 호조선일보기자 성경에서는 돈과 영은 동시에 섬길 수 없다고 했으나 영혼까지 끌어모아 돈을 추구하는 모습은 이제 이 시대의 흔한 풍경이 됐다 영을 추구하면서 동시에 돈을 추구하는 우리의 모습은 부정할 수 없는 사실이다고상한 동시에 속물적인 이 분열적인 모습을 통합하려면 돈의 길을 제대로 아는 수밖에 뉴욕 주민의 진짜 미국 식주식투자라는 직설적인 제목의 책으로 신뢰를 얻고 있는 전직 월가 트레이더 뉴욕 주민을 돈의 길의 안내자로 초대했다 뉴욕 주민은 월스트리트에서 직접 체득한 돈의 법칙을 쉬운 언어로 전달하는 독보적인 인사이트를 가진 미국 주식 교육가다 맥킨 지시티그룹 모건 등 다수의 전략 컨설팅 및 투자은행에서의 인수합병 경력을 거쳐 뉴욕 소재 헤지펀드에서 억달러 약 조원 규모의 주식형펀드를 운용하는 펀드매니저와 애널리스트로 활약했다 현재는 월가의 애널리스트와 헤지펀드 트레이 더들의 실무 트레이닝을 담당하는 교육 전문가로 일하고 있다 민족사관고등학교를 나와 월스트리트 사관학교라 불리는 펜실베이니아대와튼스쿨을 조기 졸업했다 월스트리트는 어떤 곳인 가 전쟁터 다수많은 좌절이 깔려 있고 그래서 실패를 딛고 여전히 그곳에 살아남은 자들의 매력이 형형한 곳이 다 월가를 떠나지 않은 모든 이들에겐 반드시 배울게 있다 왜 뉴욕 주민인가 최근까지 기관 헤지펀드에 있었다 직업윤리상영리활동이 안 되고 이해가 상충되는 부분도 있어서 사생활과 분리하 려고 닉네임을 쓰고 있다금융지식의 보편화라는 미션을 내건 유튜브 채널 뉴욕 주민은 현재 만 명의 구독자를 확보하고 있다 미국이나 한국이나 개미들의 움직임은 신종 코로나 바이러스 감염증 코로나으로 촉발됐다코로나 이전과 이후 격변이 있었다 주식시장은 크게 년 단위로 변화를 보인 다년닷컴 버블 년리먼 브러더스 사태년 코로나 격변큰 사건 이후로 시장 참여자들이 확 늘어난다그때마다 떼돈 벌

## Train & Test Data

In [15]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)
print(len(dataset_train))

728


In [16]:
print(len(dataset_train)) # 1344 (28945) 
print(len(dataset_test)) # 448 (9649)

728
243


## KoBERT 입력 데이터로 만들기

* 데이터를 train data와 test data로 나누었다면 각 데이터가 KoBERT 모델의 입력으로 들어갈 수 있는 형태가 되도록 토큰화, 정수 인코딩, 패딩 등을 진행

In [17]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair = pair) # 패딩과 토큰화 진행
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [18]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [19]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [20]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## KoBERT 학습모델 만들기

In [21]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=10,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [22]:
torch.cuda.is_available() # GPU 사용여부 확인

True

In [23]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5) #.to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


## KoBERT 모델 학습시키기

In [24]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long() # .to(device)
        segment_ids = segment_ids.long()# .to(device)
        valid_length= valid_length
        label = label.long() #.to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long() # .to(device)
        segment_ids = segment_ids.long() # .to(device)
        valid_length= valid_length
        label = label.long()# .to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/12 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.3274526596069336 train acc 0.125
epoch 1 train acc 0.1228298611111111


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.4075520833333333


  0%|          | 0/12 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 1.8221075534820557 train acc 0.5625
epoch 5 train acc 0.4631076388888889


  0%|          | 0/4 [00:00<?, ?it/s]

epoch 5 test acc 0.4075520833333333


## 새로운 문장 테스트

In [32]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_resume):

    data = [predict_resume, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long() # .to(device)
        segment_ids = segment_ids.long() # .to(device)

        valid_length= valid_length
        label = label.long() # .to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("대인관계능력")

            elif np.argmax(logits) == 1:
                test_eval.append("분석적 사고")

            elif np.argmax(logits) == 2:
                test_eval.append("성실성")

            elif np.argmax(logits) == 3:
                test_eval.append("열정")

            elif np.argmax(logits) == 4:
                test_eval.append("의사소통능력")

            elif np.argmax(logits) == 5:
                test_eval.append("전략적 사고")

            elif np.argmax(logits) == 6:
                test_eval.append("전문성")

            elif np.argmax(logits) == 7:
                test_eval.append("주인의식")

            elif np.argmax(logits) == 8:
                test_eval.append("자기개발")

            elif np.argmax(logits) == 9:
                test_eval.append("리더십")

        
        print(">> 입력하신 자기소개서에 드러난 직무기초능력은 " + str(test_eval) + "(으)로 판단됩니다.")


using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


* 링커리어 자기소개서 스크랩해서 직무가 일치하는지 테스트

In [33]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    resume = input("자기소개서를 입력해주세요 : ")
    if resume == 'stop' :
        break
    predict(resume)
    print("\n")

자기소개서를 입력해주세요 : 실패하더라도 최선을 다해 도전하라"제 성장 과정에 가장 큰 영향을 끼친 아버지의 가르침은 ‘실패하더라도 최선을 다해 도전하라’입니다. 아버지께서는 제게 무언가를 잘하라고 다그치지 않으셨습니다. 다만 어떤 일을 하더라도 최선을 다함으로써 후회가 남지 않도록 하라고 말씀하셨습니다. 이 가르침은 선천적으로 타고난 재능이 많지 않았던 저에게 큰 버팀목이 되었고, 끈질기게 도전해서 반드시 성과를 만들어 내는 사람으로 성장할 수 있었습니다.중학교 때 반장 선거에 떨어졌으나 고등학교 때 다시 도전하여 부반장으로 당선될 수 있었고, 대학교 동아리 활동 당시에는 감소하는 신입 부원 문제를 버스킹 홍보 활동을 기획하여 해결해냈습니다. 또한, 스타트업 인턴으로 근무할 때는 매일 1시간씩 일찍 출근하고 2시간씩 늦게 퇴근하며, 성능 개선이 이루어지지 않던 모델을 약 7％ 향상시켰습니다. 동화그룹 안에서도 주어진 문제를 반드시 해결하는 신입사원으로 거듭나겠습니다."업계 최초, 자체 개발한 스마트팩토리 플랫폼"보드 업계 최초로 자체 스마트팩토리 플랫폼을 개발한 동화그룹에서 데이터 역량을 발휘하고자 지원하였습니다. 저는 3가지 역량을 개발하여 국내 최고 데이터 엔지니어로 거듭나는 것이 목표입니다. 첫째, 대규모 데이터를 수집할 수 있는 안정적인 데이터 파이프라인을 구축하겠습니다. 스마트팩토리로부터 유입되는 다양한 데이터를 유실 없이 적재하겠습니다. 둘째, 수집한 데이터를 처리할 수 있는 전처리 능력을 키우겠습니다. 데이터가 커질수록 전처리하는 데 시간이 많이 소요되는 것을 알고 있습니다. 따라서 대규모 데이터를 분산 처리할 수 있는 역량을 키우겠습니다. 셋째, 대규모 데이터를 효율적으로 저장할 수 있는 역량을 개발하겠습니다. 클라우드 공간에 데이터 규모와 용도에 맞게 데이터를 저장할 수 있는 능력을 배양하겠습니다. 위 역량을 바탕으로 동화그룹 안에서 업계 최초이자, 최고의 스마트팩토리 플랫폼을 구축해나가겠습니다. 


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 자기소개서에 드러난 직무기초능력은 ['분석적 사고'](으)로 판단됩니다.


자기소개서를 입력해주세요 : 계속해서 달리는 인재"저를 한 단어로 표현하면 '마라톤'이라고 생각합니다. 왜냐하면, 지난 1년 반 동안 2번의 교육과 3번의 웹 프로젝트로 웹 개발 역량을 쌓아왔으며 앞으로도 계속해서 역량을 쌓기 위해 나아갈 것이기 때문입니다.좋은점 1학부 시절 졸업 프로젝트로 Java를 사용한 수화 번역기를 개발하면서 객체지향 프로그래밍에 매력을 느끼게 되었고, Java 기반 개발 방법론을 찾아보던 중 웹 개발에 대해 관심을 갖게 되었습니다.따라서 300시간의 'JAVA 프레임워크 실무 개발자 양성과정'에 참여해 웹 개발 기초를 쌓았습니다. 하지만 300시간으로는 웹을 이해하고 완성도 높은 프로젝트를 개발하는 것에 어려움이 있었습니다. 이에 저는 부족한 역량을 채우기 위해 ‘OO OO OO OOOO’에 도전하게 되었고, 1년간의 웹 개발 교육을 받을 좋은 기회를 얻게 되었습니다. 6개월 동안 Spring과 vue.js 를 사용한 프론트엔드와 백엔드 모두 학습하며 웹 개발 역량을 길렀습니다. 교육을 받고 사용자 위치정보를 기반으로 한 여행 기록 플랫폼 프로젝트와 빅데이터를 사용하여 사용자 취향을 고려한 레시피 추천 프로젝트를 진행하였고, 두 번 모두 프로젝트 발표회에서 우수상을 수여할 수 있었습니다. 프로젝트를 통해 백엔드와 프론트엔드를 모두 참여하면서 풀 스택 개발 역량을 기를 수 있었습니다. 프론트 개발에 있어서는 컴포넌트를 최대한 재사용하며 효율적으로 UI를 구성하고 코드와 파일을 간소화하여 가독성을 높였습니다. 백엔드 개발에 있어서는 기능별로 서버를 분리한 MSA 구조의 서비스를 AWS 상에 구축하여 안정적인 서비스 구축하고, MVC 구조로 효율적인 RESTfulAPI를 개발하였으며, Jenkins로 배포를 자동화하며 프로젝트 배포의 편의성을 높였습니다.이렇게 한가지의 길로 쭉 노력해온 만큼 앞으로도 계속해서 발전해나가는 인재라고 확신합니다.
>> 

* 더 많은 데이터가 필요할 듯!